In [ ]:
! sudo apt-get update
! sudo apt-get upgrade
! pip install -U scikit-learn

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sources [1,840 kB]
Get:

# CLONING AUDIO PREPROCESSING SCRIPTS

In [ ]:
! sudo rm -rf cough_sound_analysis_deep_learning
! sudo rm *.csv
! sudo rm *.txt
! sudo rm *.json
! git clone https://github.com/AdarshNandanwar/cough-sound-analysis-deep-learning.git cough_sound_analysis_deep_learning

rm: cannot remove '*.txt': No such file or directory
rm: cannot remove '*.json': No such file or directory
Cloning into 'cough_sound_analysis_deep_learning'...
remote: Enumerating objects: 440, done.
remote: Counting objects: 100% (440/440), done.
remote: Compressing objects: 100% (393/393), done.
remote: Total 785 (delta 64), reused 420 (delta 45), pack-reused 345
Receiving objects: 100% (785/785), 154.49 MiB | 26.32 MiB/s, done.
Resolving deltas: 100% (114/114), done.
Checking out files: 100% (650/650), done.


#DATASET GENERATION
NOTE- Steps 2 and 3 dosen't work in google colab
1. Add entries in the google sheet for youtube dataset
2. Clear existing youtube datasets
3. Download, crop and store the youtube audio files in directory "yt_dataset"
4. Clean the custom datasets "detection_dataset", "classification_dataset" to create "clean_detection_dataset", "clean_classification_dataset"
5. Process the detection dataset "clean_detection_dataset" and classification datasets "clean_classification_dataset", ""yt_dataset"



In [ ]:
! pip install soundfile

In [ ]:
# Running scripts to create the dataset

# Add data in the google sheet:
# https://docs.google.com/spreadsheets/d/18BCme4ZxUIGwpzmzTnCSsAxV3kOm_OH8yAHJlDRDDMQ/edit?usp=sharing

# Doesn't work on colab
# ! python3 cough_sound_analysis_deep_learning/clear_yt_dataset.py

# Doesn't work on colab
# ! python3 cough_sound_analysis_deep_learning/download_yt_dataset.py

! python3 cough_sound_analysis_deep_learning/clean_dataset.py

! python3 cough_sound_analysis_deep_learning/audio_preprocessing.py

Creating /content/cough_sound_analysis_deep_learning/clean_detection_dataset
Done!
Creating /content/cough_sound_analysis_deep_learning/clean_classification_dataset
Done!
Creating detection feature dataset.
Detection feature dataset created successuflly.
Creating classification feature dataset.
/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate 

# SELECTING FEATURES
**Include all features**- [:, :-1]
<br> **Include some features**: n = 216 (for 5 sec duration at sr = 22050)
<br> mfcc- [:, n*0:n*13]
<br> spectral centroids- [:, n*13:n*14]
<br> spectral rolloffs- [:, n*14:n*15]
<br> spectral bandwidth 2- [:, n*15:n*16]
<br> spectral bandwidth 3- [:, n*16:n*17]
<br> spectral bandwidth 4- [:, n*17:n*18]
<br> zero crossing rates- [:, n*18:n*19]
<br> chroma features- [:, n*19:n*31]


In [ ]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

# Importing the classification dataset
classification_dataset = pd.read_csv('classification_data.csv', header=None)
classification_dataset = shuffle(classification_dataset)
classification_dataset.reset_index(inplace=True,drop=True)
# Include all features
X_classification = classification_dataset.iloc[:, :-1]
y_classification = classification_dataset.iloc[:, -1]
print("classification dataset")
print(X_classification.shape)
print(y_classification.shape)

# Importing the detection dataset
detection_dataset_1 = pd.read_csv('detection_data.csv', header=None)
# Merging classification dataset
detection_dataset_2 = classification_dataset.copy()
detection_dataset_2.iloc[:, -1] = 1
detection_dataset = pd.concat([detection_dataset_1, detection_dataset_2], ignore_index=True)
detection_dataset = shuffle(detection_dataset)
detection_dataset.reset_index(inplace=True,drop=True)
# Include all features
X_detection = detection_dataset.iloc[:, :-1]
y_detection = detection_dataset.iloc[:, -1]
print("detection dataset")
print(X_detection.shape)
print(y_detection.shape)


classification dataset
(412, 6696)
(412,)
detection dataset
(678, 6696)
(678,)


In [ ]:
print(detection_dataset.iloc[:,-1].value_counts())
print(classification_dataset.iloc[:,-1].value_counts())


1    442
0    236
Name: 6696, dtype: int64
1    148
0    131
3     70
2     63
Name: 6696, dtype: int64


In [ ]:
# Getting the feature vector for prediction

from cough_sound_analysis_deep_learning.audio_preprocessing import get_features_csv_row
import librosa
SAMPLE_RATE = 22050

file_path = "/content/cough_sound_analysis_deep_learning/detection_dataset/cough/1-19111-A-24.wav"
signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
features = get_features_csv_row(signal, sample_rate)
print(len(features))

6696


# DETECTION MODEL




In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder , StandardScaler 

detection_dataset = pd.read_csv('./detection_data.csv')
X_detection = detection_dataset.iloc[:, :-1]
y_detection = detection_dataset.iloc[:, -1]
 
encoder = LabelEncoder()
y = encoder.fit_transform(y_detection)
scaler = StandardScaler()
X = scaler.fit_transform(np.array(X_detection,dtype=float))
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

from IPython.display import clear_output

In [ ]:

def best_model(X,y):

  model_data = {}
  train_accuracies = []

  print('#################RANDOM FOREST CLASSIFIER########################')
  rfc = RandomForestClassifier(random_state=22)
  param_grid = { 
      'n_estimators': [200,400],
      'max_features': ['auto', 'sqrt', 'log2'],
      'max_depth' : [4,5,6,7,8],
      'criterion' :['gini', 'entropy']
  }
  gs_rfc = GridSearchCV(estimator=rfc,param_grid=param_grid,cv=5,scoring='accuracy',verbose=3)
  gs_rfc.fit(X,y)
  print('Done')
  model_data['rfc'] = gs_rfc.best_params_
  train_accuracies.append(gs_rfc.best_score_)

  print('###################LOGISTIC REGRESSION##########################')
  lr = LogisticRegression()
  params = {
      'C':[0.01,0.1],
      'max_iter':[500,700],
      'solver':['sag','saga','lbfgs','liblinear'],
      'penalty':['l2','l1']   
  }
  gs_lr = GridSearchCV(estimator=lr,param_grid=params,verbose=3,scoring='accuracy',cv=5)
  gs_lr.fit(X,y)
  print('Done')
  model_data['lr'] = gs_lr.best_params_
  train_accuracies.append(gs_lr.best_score_)

  print('##############################SVM###############################')
  svm = SVC()
  params = {
      'C':[0.01,0.1],
      'kernel':['linear','poly','rbf'],
      'gamma':['scale','auto']
  }
  gs_svm = GridSearchCV(estimator=svm,param_grid=params,verbose=3,scoring='accuracy',cv=5)
  gs_svm.fit(X,y)
  print('Done')
  model_data['svm'] = gs_svm.best_params_
  train_accuracies.append(gs_svm.best_score_)

  print('#######################Kth NEAREST NEIGHBOR####################')
  knn = KNeighborsClassifier()
  params = {
      'n_neighbors':[5,7,10,13,16],
      'weights':['uniform','distance'],
      'algorithm':['ball_tree','kd_tree'],
  }
  gs_knn = GridSearchCV(estimator=knn,param_grid=params,verbose=3,scoring='accuracy',cv=5)
  gs_knn.fit(X,y)
  print('Done')
  model_data['knn'] = gs_knn.best_params_
  train_accuracies.append(gs_knn.best_score_)

  print('####################GAUSIAN NB###############################')
  gnb= GaussianNB()
  gnb.fit(X_train,y_train)
  model_data['gnb'] = 'no params'
  train_accuracies.append(accuracy_score(y,gnb.predict(X)))

  return model_data, train_accuracies



In [ ]:
model_data, train_accuracies = best_model(X_train,y_train)
clear_output()
print('Train accuracies : ')
print(train_accuracies)
print(model_data)

#################RANDOM FOREST CLASSIFIER########################
Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] criterion=gini, max_depth=4, max_features=auto, n_estimators=200 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  criterion=gini, max_depth=4, max_features=auto, n_estimators=200, score=0.919, total=   0.7s
[CV] criterion=gini, max_depth=4, max_features=auto, n_estimators=200 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


[CV]  criterion=gini, max_depth=4, max_features=auto, n_estimators=200, score=0.892, total=   0.8s
[CV] criterion=gini, max_depth=4, max_features=auto, n_estimators=200 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.5s remaining:    0.0s


[CV]  criterion=gini, max_depth=4, max_features=auto, n_estimators=200, score=0.865, total=   0.8s
[CV] criterion=gini, max_depth=4, max_features=auto, n_estimators=200 
[CV]  criterion=gini, max_depth=4, max_features=auto, n_estimators=200, score=0.919, total=   0.7s
[CV] criterion=gini, max_depth=4, max_features=auto, n_estimators=200 
[CV]  criterion=gini, max_depth=4, max_features=auto, n_estimators=200, score=0.892, total=   0.7s
[CV] criterion=gini, max_depth=4, max_features=auto, n_estimators=400 
[CV]  criterion=gini, max_depth=4, max_features=auto, n_estimators=400, score=0.919, total=   1.5s
[CV] criterion=gini, max_depth=4, max_features=auto, n_estimators=400 
[CV]  criterion=gini, max_depth=4, max_features=auto, n_estimators=400, score=0.892, total=   1.5s
[CV] criterion=gini, max_depth=4, max_features=auto, n_estimators=400 
[CV]  criterion=gini, max_depth=4, max_features=auto, n_estimators=400, score=0.865, total=   1.5s
[CV] criterion=gini, max_depth=4, max_features=auto

In [ ]:
model1 = RandomForestClassifier(criterion=model_data['rfc']['criterion'],
                                max_depth=model_data['rfc']['max_depth'],
                                max_features=model_data['rfc']['max_features'],
                                n_estimators=model_data['rfc']['n_estimators'])
model2 = LogisticRegression(C=model_data['lr']['C'],
                            max_iter=model_data['lr']['max_iter'],
                            penalty=model_data['lr']['penalty'],
                            solver=model_data['lr']['solver'])
model3 = SVC(C=model_data['svm']['C'],
             gamma=model_data['svm']['gamma'],
             kernel=model_data['svm']['kernel'])
model4 = KNeighborsClassifier(algorithm=model_data['knn']['algorithm'],
                              n_neighbors=model_data['knn']['n_neighbors'],
                              weights=model_data['knn']['weights'])
model5 = GaussianNB()

model1.fit(X_train,y_train)
model2.fit(X_train,y_train)
model3.fit(X_train,y_train)
model4.fit(X_train,y_train)
model5.fit(X_train,y_train)

models = [model1,model2,model3,model4,model5]

test_accuracies = []
test_accuracies.append(accuracy_score(y_test,model1.predict(X_test)))
test_accuracies.append(accuracy_score(y_test,model2.predict(X_test)))
test_accuracies.append(accuracy_score(y_test,model3.predict(X_test)))
test_accuracies.append(accuracy_score(y_test,model4.predict(X_test)))
test_accuracies.append(accuracy_score(y_test,model5.predict(X_test)))
print(train_accuracies)
index = test_accuracies.index(max(test_accuracies))
models[index]
print(index)
final_detection_model = models[index]

[0.9081081081081083, 0.9081081081081083, 0.9081081081081083, 0.9081081081081083, 0.8594594594594595]


# CLASSIFICATION MODEL


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder , StandardScaler 


In [ ]:
# Reading Data from csv file
classification_dataset = pd.read_csv('./classification_data.csv',header=None)
# Getting X and y
X_classification = classification_dataset.iloc[:, :-1]
y_classification = classification_dataset.iloc[:, -1]
# Scaling and Encoding
encoder = LabelEncoder()
y = encoder.fit_transform(y_classification)
scaler = StandardScaler()
X = scaler.fit_transform(np.array(X_classification,dtype=float))
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
# from sklearn.externals import joblib

from IPython.display import clear_output

In [ ]:
def find_best_model(X,y):
  model_data = {}
  train_accuracies = []
  
  print('####################RANDOM FOREST CLASSIFIER ###################')
  rfc = RandomForestClassifier()  
  params = {
      'n_estimators':[300,500],
      'max_depth':[5,7,9],
      'max_features': ['auto', 'sqrt', 'log2'],
      'criterion' :['gini', 'entropy']
  }
  gs_rfc = GridSearchCV(estimator=rfc,param_grid=params,verbose=3,scoring='accuracy',cv=5)
  gs_rfc.fit(X,y)
  print('Done')
  print('Best Train Accuracy : ')
  print( gs_rfc.best_score_)
  model_data['rfc'] = gs_rfc.best_params_
  train_accuracies.append(gs_rfc.best_score_)

  print('##################Kth NEIGHBORS CLASSIFIER######################')
  knn = KNeighborsClassifier()
  params = {
      'n_neighbors':[5,7],
      'weights':['uniform','distance'],
      'algorithm':['auto','ball_tree','kd_tree'],
      'leaf_size':[30,50]      
  }
  gs_knn = GridSearchCV(estimator=knn,param_grid=params,verbose=3,scoring='accuracy',cv=5)
  gs_knn.fit(X,y)
  print('Done')
  print('Best Train Accuracy : ')
  print(gs_knn.best_score_)
  model_data['knn'] = gs_knn.best_params_
  train_accuracies.append(gs_knn.best_score_)
  
  print('########################Linear SVC#############################')
  lsvc = LinearSVC(dual=True)
  params = {
      'penalty':['l1','l2'],
      'C':[0.01,0.1],
      'loss':['hinge','squared_hinge']
  }
  gs_lsvc = GridSearchCV(estimator=lsvc,param_grid=params,verbose=3,scoring='accuracy',cv=5)
  gs_lsvc.fit(X,y)
  print('Done')
  print('Best Train Accuracy : ')
  print(gs_lsvc.best_score_)
  model_data['lsvc'] = gs_lsvc.best_params_
  train_accuracies.append(gs_lsvc.best_score_)

  print('#######################MLP CLASSIFIER#########################')
  mlp = MLPClassifier()
  params = {
      'activation':['tanh','relu'],
      'learning_rate':['constant','invscaling','adaptive'],
  }
  gs_mlp = GridSearchCV(estimator=mlp,param_grid=params,verbose=3,scoring='accuracy',cv=5)
  gs_mlp.fit(X,y)
  print('Done')
  print('Best Train Accuracy : ')
  print(gs_mlp.best_score_)
  model_data['mlp'] = gs_mlp.best_params_
  train_accuracies.append(gs_mlp.best_score_)


  return model_data, train_accuracies



In [ ]:
model_data, train_accuracies = find_best_model(X_train,y_train)
clear_output()
print(model_data)
print('Train Accuracies : ')
print(train_accuracies)

{'rfc': {'criterion': 'entropy', 'max_depth': 9, 'max_features': 'log2', 'n_estimators': 500}, 'knn': {'algorithm': 'auto', 'leaf_size': 30, 'n_neighbors': 5, 'weights': 'distance'}, 'lsvc': {'C': 0.01, 'loss': 'squared_hinge', 'penalty': 'l2'}, 'mlp': {'activation': 'relu', 'learning_rate': 'adaptive'}}
Train Accuracies : 
[0.941016333938294, 0.7777979431336963, 0.8475499092558983, 0.9237144585601935]


In [ ]:
# Finding test accuracies and choosing best model
test_accuracies = []

model1 = RandomForestClassifier(criterion=model_data['rfc']['criterion'],
                                max_depth=model_data['rfc']['max_depth'],
                                max_features=model_data['rfc']['max_features'],
                                n_estimators=model_data['rfc']['n_estimators'])
model2 = KNeighborsClassifier(algorithm=model_data['knn']['algorithm'],
                              leaf_size=model_data['knn']['leaf_size'],
                              n_neighbors=model_data['knn']['n_neighbors'],
                              weights=model_data['knn']['weights'])
model3 = LinearSVC(dual=True,
                   C=model_data['lsvc']['C'],
                   loss=model_data['lsvc']['loss'],
                   penalty=model_data['lsvc']['penalty'])
model4 = MLPClassifier(activation=model_data['mlp']['activation'],
                       learning_rate=model_data['mlp']['learning_rate'])

model1.fit(X_train,y_train)
model2.fit(X_train,y_train)
model3.fit(X_train,y_train)
model4.fit(X_train,y_train)

models = [model1,model2,model3,model4]

test_accuracies.append(accuracy_score(y_test,model1.predict(X_test)))
test_accuracies.append(accuracy_score(y_test,model2.predict(X_test)))
test_accuracies.append(accuracy_score(y_test,model3.predict(X_test)))
test_accuracies.append(accuracy_score(y_test,model4.predict(X_test)))

print(test_accuracies)

max_index = test_accuracies.index(max(test_accuracies))
final_classification_model = models[max_index]

print(max_index)

[0.9516129032258065, 0.782258064516129, 0.8145161290322581, 0.9193548387096774]
0


# SAVING TRAINED MODELS

In [ ]:
from google.colab import files
files.download('./detection_data.csv')
files.download('./classification_data.csv')

In [ ]:
# save the model to disk

import pickle

# Detection model
filename = 'detection_model.sav'
pickle.dump(final_detection_model, open(filename, 'wb'))

# Classification model
filename = 'classification_model.sav'
pickle.dump(final_classification_model, open(filename, 'wb'))

from google.colab import files
files.download('./detection_model.sav')
files.download('./classification_model.sav')